In [62]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, RepeatVector, concatenate
from keras.preprocessing.sequence import pad_sequences
import utils
from sklearn.model_selection import train_test_split
import numpy as np

In [63]:
CLEANED_ARTICLES_FILE = '../data/cleaned_articles_ed.txt'
CLEANED_HEADLINES_FILE = '../data/cleaned_headlines_ed.txt'

In [64]:
with open(CLEANED_ARTICLES_FILE,'r', encoding="utf-8") as file:
    data = file.read()
    articles = data.split('\n')     
    
with open(CLEANED_HEADLINES_FILE,'r', encoding="utf-8") as file:
    data = file.read()
    headlines = data.split('\n') 

In [65]:
art_tokenizer, encoded_art = utils.create_tokenizer(articles)
head_tokenizer, encoded_head = utils.create_tokenizer(headlines)

In [66]:
MAX_LENGTH = 60
MAX_HEADLINE_LENGTH = 20

encoder_vocab_size = len(art_tokenizer.word_index)
decoder_vocab_size = len(head_tokenizer.word_index)

print(encoder_vocab_size)
print(decoder_vocab_size)
print(src_txt_length)
print(sum_txt_length)

src_txt_length = len(encoded_art)
sum_txt_length = len(encoded_head)

padded_encoded_art = pad_sequences(encoded_art,  maxlen=src_txt_length, padding='post')
padded_encoded_head = pad_sequences(encoded_head,  maxlen=sum_txt_length, padding='post')

16586
5017
2692
2692


In [67]:
X_train, X_test, y_train, y_test = train_test_split(np.array(padded_encoded_art), np.array(padded_encoded_head), test_size=.20, random_state=82)

In [68]:
#https://machinelearningmastery.com/encoder-decoder-models-text-summarization-keras/

#I would choose Alternate 3 from this link
#That I belive is how we should create our encoder deccoder model

#NOTE: Example code here
# article input model
inputs1 = Input(shape=(src_txt_length,))
article1 = Embedding(encoder_vocab_size, 128)(inputs1)
article2 = LSTM(128)(article1)
article3 = RepeatVector(sum_txt_length)(article2)
# summary input model
inputs2 = Input(shape=(sum_txt_length,))
summ1 = Embedding(encoder_vocab_size, 128)(inputs2)


# decoder model
decoder1 = concatenate([article3, summ1])
decoder2 = LSTM(128)(decoder1)
outputs = Dense(decoder_vocab_size, activation='softmax')(decoder2)

# tie it together [article, summary] [word]
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 2692)]               0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, 2692, 128)            2123008   ['input_5[0][0]']             
                                                                                                  
 lstm_4 (LSTM)               (None, 128)                  131584    ['embedding_4[0][0]']         
                                                                                                  
 input_6 (InputLayer)        [(None, 2692)]               0         []                            
                                                                                            

In [69]:
#https://medium.com/@duncanboynton/seq2seq-news-article-summary-using-an-encoder-decoder-lstm-to-summarize-text-5de56fccfbf6


In [70]:
model.fit([X_train, y_train], y_train, epochs=4, verbose=1)

Epoch 1/4


ValueError: in user code:

    File "C:\Users\hsubr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\hsubr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\hsubr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\hsubr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line